In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Load the simulated transaction dataset.**

In [2]:
data = {
    'CustomerID': [1, 2, 1, 3, 2, 4, 5, 4, 3, 5, 1, 2],
    'OrderDate': [
        '2023-01-01', '2023-01-05', '2023-02-01', '2023-02-10', '2023-02-15',
        '2023-03-01', '2023-03-05', '2023-03-10', '2023-03-15', '2023-04-01',
        '2023-04-05', '2023-04-10'
    ],
    'OrderValue': [100, 200, 150, 300, 250, 400, 500, 450, 350, 600, 700, 550]
}

# Create a DataFrame
df = pd.DataFrame(data)

In [4]:
df = pd.DataFrame(data)
df

,CustomerID,OrderDate,OrderValue
0,1,2023-01-01,100
1,2,2023-01-05,200
2,1,2023-02-01,150
3,3,2023-02-10,300
4,2,2023-02-15,250
5,4,2023-03-01,400
6,5,2023-03-05,500
7,4,2023-03-10,450
8,3,2023-03-15,350
9,5,2023-04-01,600


**Prepare the dataset by converting date columns to datetime format and  **

**calculating the recency, frequency, and monetary value for each customer **

In [5]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# Calculate the total revenue (monetary value) for each customer
monetary = df.groupby('CustomerID')['OrderValue'].sum().reset_index()
monetary.columns = ['CustomerID', 'Monetary']

# Calculate the frequency of transactions for each customer
frequency = df.groupby('CustomerID')['OrderDate'].count().reset_index()
frequency.columns = ['CustomerID', 'Frequency']

# Calculate the recency for each customer (the number of days since the last purchase)
max_date = df['OrderDate'].max()
recency = df.groupby('CustomerID')['OrderDate'].max().reset_index()
recency.columns = ['CustomerID', 'LastPurchaseDate']
recency['Recency'] = (max_date - recency['LastPurchaseDate']).dt.days

# Merge all data into a single DataFrame
rfm = pd.merge(recency, frequency, on='CustomerID')
rfm = pd.merge(rfm, monetary, on='CustomerID')


Calculate the 3-month CLV for each customer using average monthly revenue

multiplied by the number of months

In [8]:
# Calculate the number of months each customer has been active
rfm['MonthsActive'] = (df.groupby('CustomerID')['OrderDate']
                       .min()
                       .apply(lambda x: (max_date - x).days / 30)
                       .reset_index(drop=True))

# Average monthly revenue per customer
rfm['AvgMonthlyRevenue'] = rfm['Monetary'] / rfm['MonthsActive']

# 3-month CLV estimation
rfm['CLV_3_Months'] = rfm['AvgMonthlyRevenue'] * 3

# Display results
print(rfm[['CustomerID', 'CLV_3_Months']])

   CustomerID  CLV_3_Months
0           1    863.636364
1           2    947.368421
2           3    991.525424
3           4   1912.500000
4           5   2750.000000


In [9]:
rfm['average_order_value'] = rfm['Monetary'] / rfm['Frequency']
rfm


,CustomerID,LastPurchaseDate,Recency,Frequency,Monetary,average_order_value,MonthsActive,AvgMonthlyRevenue,CLV_3_Months
0,1,2023-04-05,5,3,950,316.666667,3.300000,287.878788,863.636364
1,2,2023-04-10,0,3,1000,333.333333,3.166667,315.789474,947.368421
2,3,2023-03-15,26,2,650,325.000000,1.966667,330.508475,991.525424
3,4,2023-03-10,31,2,850,425.000000,1.333333,637.500000,1912.500000
4,5,2023-04-01,9,2,1100,550.000000,1.200000,916.666667,2750.000000
